# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/aqsa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aqsa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [7]:
!mkdir data

mkdir: data: File exists


In [8]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31524    0 31524    0     0  54129      0 --:--:-- --:--:-- --:--:-- 54072


In [9]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70549    0 70549    0     0   592k      0 --:--:-- --:--:-- --:--:--  593k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [10]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [11]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [12]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [13]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/24 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 13, relationships: 46)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 13, relationships: 46)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


#### ❓ Answer #1:

1) Single Hop Specific Query: Goes to only one place to answer the query (Para says Albert Was born IN Jamaica. If the question is Where was Albert born? It answers using the single hop):  Focused on one KG edge. A 
A typical usecase is Baseline factual QA; helpful for grounding the system on entity-property-value relations.

2) Multi Hop Abstract Query: Multiple sources(i.e Involves multi-edge reasoning, it needs to traverse to multiple edges to answer the question). Also, it will answer a abstract question, but not a simple conceptual question(not a fact). ie answer harder/open ended questions. It does so by connecting multiple facts that may not be tied to specific entities. Usecase include  stress-test multi-hop reasoning without giving away paths explicitly.

3) Multi Hop Specific Query: Multiple sources/chunks, and answers a factual question. Ie. It generates multi-hop questions that are detailed and grounded in specific entities. Usecase includes Evaluating systems' ability to track and chain factual knowledg

The second parameter is assigning weights distribution to the synthesize ( 50% simple factual (Single-Hop), 25% thematic, hard general reasoning (Multi-Hop Abstract), 25% hard, but specific reasoning (Multi-Hop Specific)) 
This ensures Breadth: Variety in question types; Depth: Evaluates system's single-hop vs multi-hop reasoning; Realism: Mimics real-world query distribution (some simple, some vague, some difficult)

Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What role does OpenAI play in the development ...,[The ethics of this space remain diabolically ...,"According to the provided context, OpenAI was ...",single_hop_specifc_query_synthesizer
1,What is the role of Artifical Intelligance in AI?,[Simon Willison’s Weblog Subscribe Stuff we fi...,Artificial Intelligence is the academic field ...,single_hop_specifc_query_synthesizer
2,How are Large Language Models (LLMs) impacting...,[the document includes some of the clearest ex...,The context discusses the profound impact of L...,single_hop_specifc_query_synthesizer
3,What is Mistral in the context of AI models?,"[The environmental impact got much, much worse...",Mistral is listed among the organizations with...,single_hop_specifc_query_synthesizer
4,What is Qwen2-VL and how does it relate to mul...,"[and generic, but my phone can pitch bland and...",Qwen2-VL is mentioned as part of the recent tr...,single_hop_specifc_query_synthesizer
5,how GPT-4 breakthroughs compare to current AI ...,[<1-hop>\n\nThe ethics of this space remain di...,The context shows that despite the limitations...,multi_hop_abstract_query_synthesizer
6,how much cost training llms and is it worth it...,[<1-hop>\n\nThe ethics of this space remain di...,The context explains that training large langu...,multi_hop_abstract_query_synthesizer
7,how does the environmental impact of ai and lm...,"[<1-hop>\n\nThe environmental impact got much,...",the context shows that the environmental impac...,multi_hop_abstract_query_synthesizer
8,Considering the significant advancements and k...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,The developments in 2024 highlight a rapidly e...,multi_hop_specific_query_synthesizer
9,How do the recent advancements in voice and vi...,[<1-hop>\n\nthe document includes some of the ...,The context highlights that models like Google...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/18 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is EleutherAI and how does it relate to t...,[The ethics of this space remain diabolically ...,"According to the provided context, EleutherAI ...",single_hop_specifc_query_synthesizer
1,AI is what in 2023?,[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, AI refers to Large Language Models (L...",single_hop_specifc_query_synthesizer
2,What is OpenAI in context of large language mo...,[the document includes some of the clearest ex...,The context mentions OpenAI as one of the top ...,single_hop_specifc_query_synthesizer
3,Wha is OpenAI's role in the development of lar...,[Things we learned about LLMs in 2024 31st Dec...,The context does not provide specific informat...,single_hop_specifc_query_synthesizer
4,"How do AI model release strategies, like Qwen ...",[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,The document discusses how effective AI model ...,multi_hop_abstract_query_synthesizer
5,How does the dependence of agent capabilities ...,[<1-hop>\n\nacknowledge that. If you tell me t...,The context highlights that the dependence of ...,multi_hop_abstract_query_synthesizer
6,How does the importance of training data quali...,[<1-hop>\n\nthis. LLMs need better criticism A...,The context emphasizes that the most critical ...,multi_hop_abstract_query_synthesizer
7,How do the recent advancements in Large Langua...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,Recent advancements in Large Language Models (...,multi_hop_abstract_query_synthesizer
8,How does Google's recent advancements in large...,[<1-hop>\n\nThings we learned about LLMs in 20...,"In 2024, Google made significant strides in la...",multi_hop_specific_query_synthesizer
9,"Based on Simon Willison’s insights, how do the...",[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,Simon Willison’s weblog highlights that 2023 w...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [23]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years _ Homework Attempt 2!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [24]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [25]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [27]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [28]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [29]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [30]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [32]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [33]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague and unclear term in the AI community. Generally, it refers to AI systems that can go away and act on your behalf, often imagined as digital assistants or travel-agent-like models. Another interpretation involves large language models (LLMs) given access to tools which they can run iteratively to solve problems. However, there is no single, clear, or widely understood definition of "agents," and their practical, real-world deployment remains limited, partly due to challenges like gullibility in AI systems. Overall, agents are conceptually AI entities intended to act autonomously for users, but the exact meaning varies and remains unsettled.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [34]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [35]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: LangChain's evaluation module provides evaluators you can use as-is for common evaluation scenarios. qa_evaluator is one of the out-of-the-box ones. It outputs correctness of answer. i.e accuracy . Uses question + model prediction + reference answer.
- `labeled_helpfulness_evaluator`: This is another out of the box evaluator. The output Depends on criteria key, here we use helpfulness criteria( (semantic alignment + usefulness), which is a default implemented criteria. It Measures how helpful the model's output is, based on a labeled reference answer. Other criteria include conciseness, relevance, correctness, coherence, harmfulness, maliciousness. Or you can define your own criteria. 
helpfulness uses 
• input = question
• prediction = model answer
• reference = correct answer
- `dope_or_nope_evaluator`: It is using custom criteria that is Subjective, creative-style evaluator asking whether the output is cool or impressive.Uses only model output. 

## LangSmith Evaluation

In [36]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'spotless-reason-69' at:
https://smith.langchain.com/o/4e223e9d-b789-4c00-8d16-32ad70974f10/datasets/47b2c106-3f8c-45b4-80e7-b824cdc3256d/compare?selectedSessions=062e76a1-075c-4a06-9608-356ad1e41d18




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Based on the insights about Apple’s MLX librar...,"Based on the provided context, Apple’s approac...",None,The context highlights that Apple’s MLX librar...,1,1,0,9.254166,b4a68595-c6c6-4a0f-b000-626e1e621a7a,e361ac57-0ddc-41f0-8f72-db5771225344
1,H0w do LLMs sh0w the progress in AI in 2023 an...,Based on the provided context:\n\nLLMs demonst...,None,The context indicates that 2023 was a breakthr...,1,1,0,5.629479,4f8557f6-cea9-4ac6-b34c-e90358edd5c1,47a2e140-f7aa-4d06-999a-2398741ba8d1
2,"Based on Simon Willison’s insights, how do the...",Based on Simon Willison’s insights from the pr...,None,Simon Willison’s weblog highlights that 2023 w...,1,1,0,5.877384,43eeeb7a-d943-46da-8219-bfdedddf796a,2d8685bd-bb09-470f-9b5c-bbbdcbd1cf70
3,How does Google's recent advancements in large...,"Based on the provided context, Google's Gemini...",None,"In 2024, Google made significant strides in la...",1,1,0,8.948981,cf45d9a6-7a45-4dbb-879e-fb9db1a49d16,722bd858-f4b1-4d34-b173-53a831c444e5
4,How do the recent advancements in Large Langua...,Recent advancements in Large Language Models (...,None,Recent advancements in Large Language Models (...,1,1,0,4.675432,05d67017-1262-4a68-8cb4-0f967b6753be,3ccb23f2-f989-460f-b30c-e952f73492e1
5,How does the importance of training data quali...,"Based on the provided context, the quality and...",None,The context emphasizes that the most critical ...,1,1,0,3.661058,6e97d914-1b07-43b9-a9cf-be828d739f83,eb7b0d50-10af-4c7c-8d70-d5fb9ecf0431
6,How does the dependence of agent capabilities ...,The context suggests that current AI agents' c...,None,The context highlights that the dependence of ...,1,1,0,5.315406,b91c472d-bbe9-4ea8-b391-52b6dfc8a94e,aa2444be-5df9-4c89-8d15-e9b4d4b1caca
7,"How do AI model release strategies, like Qwen ...","Based on the provided context, AI model releas...",None,The document discusses how effective AI model ...,0,0,0,4.819824,205f90f1-94b5-439c-9f16-6d2e2523267e,78f8a1ba-a22f-4b1c-a292-2f4258f4ebac
8,Wha is OpenAI's role in the development of lar...,"Based on the provided context, OpenAI played a...",None,The context does not provide specific informat...,0,0,0,5.735545,4bd0440b-57d5-4629-917f-2115ac23ed41,d539adb0-9809-4d2b-8ad9-711d689980ba
9,What is OpenAI in context of large language mo...,"OpenAI is an organization that, a year ago, ha...",None,The context mentions OpenAI as one of the top ...,1,1,0,3.272184,108bd2c7-c4bd-4852-9037-2f20d940df5d,e6bd0135-8337-4ebc-8375-f4cc361ce221


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [37]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [38]:
rag_documents = docs

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

The chunk size controls how much content each chunk holds, usually measured in tokens or characters. Chunk size affects the answers accuracy, retrieval relevance, the latency and performance, affects token window management and semantic chunk boundaries (repeat vs exclusive content)

here we increse the chunk size to 1000 from 500,while keeping chunk_overlap the same. Larger chunks = bigger retrieval payloads = longer LLM context windows needed, means also increased token size and inference costs. Since most models have limited token size, Large chunks can crowd out other relevant chunks if context space is limited. Too-large chunks may introduce distractors, hurting the factuality of the generated output. However, larger chunks are Better for semantic cohesion and multi-sentence reasoning, but run the Risk of including irrelevant content (semantic drift). In other words, Chunk size directly modulates the precision, completeness, and efficiency of retrieval — and thus the factuality and helpfulness of our RAG outputs.

In [40]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

Embedding model transforms the data/content into numerical vector representation, that we then extract the chunks during retrival phase. The entire retrieval performance hinges on the embedding model’s ability to semantically match inputs. 
1) Semantic similarity : A weak embedding model will Poorly capture of meaning → irrelevant chunks. While  Strong Embedding Model leads to Precise chunk matching
2) Recall of relevant docs: A weak model Misses key info, while strong model has High precision + recall
3) Multi-hop reasoning: A weak mode has Low signal chaining, while a good model has Better chain coherence
4) Context ranking :A weak model has noisy results. strong model focuses on top-k retrieval
5) Downstream LLM quality:  A weak LLM gets wrong context → hallucinations exist. While a good LLM grounded in right context has better factual accuracy and faithfulness. 

In [41]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [42]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [43]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [44]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

"Yo, so here’s the lowdown on “agents” straight from the AI street: the term “agents” is hella vague and kinda frustrating 'cause nobody’s really nailed down what it *actually* means. Some folks see agents like your classic travel agent—something that goes out and acts on your behalf. Others think of agents as LLMs strapped with tools, running in loops to crack problems. But here's the kicker: these agents aren’t really hitting the scene yet, mostly ‘cause they fall prey to gullibility—like they just trust anything thrown their way.\n\nSo, agents are basically AI systems that *should* act autonomously for you, but the tech’s still stumbling on the truth-detection grind, meaning you gotta wait a bit longer before you get a legit, dependable AI sidekick. They’re “coming soon,” but not quite here to sling dope moves just yet.\n\nStay tuned, ‘cause until these models get their skepticism game tight—like an AGI-level hustle—we’re stuck in agent limbo. Peace!"

Finally, we can evaluate the new chain on the same test set!

In [45]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'ample-stew-18' at:
https://smith.langchain.com/o/4e223e9d-b789-4c00-8d16-32ad70974f10/datasets/47b2c106-3f8c-45b4-80e7-b824cdc3256d/compare?selectedSessions=f27d4ee2-021b-48d5-97bb-773ef0c88729




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Based on the insights about Apple’s MLX librar...,"Alright, here’s the lowdown, fresh and fly:\n\...",None,The context highlights that Apple’s MLX librar...,1,1,1,6.747858,b4a68595-c6c6-4a0f-b000-626e1e621a7a,67c418ce-3e15-4312-ba30-60569b710da5
1,H0w do LLMs sh0w the progress in AI in 2023 an...,"Aight, here’s the lowdown on LLMs flexin’ the ...",None,The context indicates that 2023 was a breakthr...,1,1,1,8.790863,4f8557f6-cea9-4ac6-b34c-e90358edd5c1,eb691afb-6f67-4f67-a80a-00a209c8d516
2,"Based on Simon Willison’s insights, how do the...","Yo, here’s the lowdown from Simon Willison’s s...",None,Simon Willison’s weblog highlights that 2023 w...,1,1,1,7.407839,43eeeb7a-d943-46da-8219-bfdedddf796a,4023ddd5-029b-423f-80ec-08a55fa2dbe7
3,How does Google's recent advancements in large...,"Alright, here’s the lowdown, straight fire sty...",None,"In 2024, Google made significant strides in la...",1,1,1,7.081409,cf45d9a6-7a45-4dbb-879e-fb9db1a49d16,9b7403df-1675-44c5-9557-bbe97271a584
4,How do the recent advancements in Large Langua...,"Yo, here’s the lowdown—LLMs are flexing hard i...",None,Recent advancements in Large Language Models (...,1,1,1,4.230204,05d67017-1262-4a68-8cb4-0f967b6753be,cc715e10-6429-4f40-9995-fbfec322e3b2
5,How does the importance of training data quali...,"Aight, here’s the lowdown—training data qualit...",None,The context emphasizes that the most critical ...,1,1,1,5.570817,6e97d914-1b07-43b9-a9cf-be828d739f83,245510d9-24eb-4da1-81ba-a02597eeb3b4
6,How does the dependence of agent capabilities ...,"Alright, so here’s the lowdown: The power of A...",None,The context highlights that the dependence of ...,1,1,1,4.243256,b91c472d-bbe9-4ea8-b391-52b6dfc8a94e,3acfff67-e9a6-43ca-abd8-b1dbde4359bf
7,"How do AI model release strategies, like Qwen ...","Yo, check it—based on the info at hand, the re...",None,The document discusses how effective AI model ...,1,1,1,6.354277,205f90f1-94b5-439c-9f16-6d2e2523267e,abae310a-76f6-47ba-8a0a-ee819b93ffe9
8,Wha is OpenAI's role in the development of lar...,"Yo, based on the vibe from the context, OpenAI...",None,The context does not provide specific informat...,0,0,1,4.834875,4bd0440b-57d5-4629-917f-2115ac23ed41,eab34bb5-87b1-4691-8cda-4ce2b96c3394
9,What is OpenAI in context of large language mo...,"Alright, here’s the lowdown on OpenAI in the l...",None,The context mentions OpenAI as one of the top ...,1,1,1,3.307975,108bd2c7-c4bd-4852-9037-2f20d940df5d,25dc17c5-9c3a-48fe-8215-714275d79fd2


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.


Answer
We noted that all three metrics improved with the lastest run, specifically dopeness with the most improvement. We used a better model(text-embedding-3-large (OpenAI’s top-tier dense retriever)) , larger chunk size and defined Dope augmentation more specifically. 

1) Correctness: Improved slightly, as text-embedding-3-large has state-of-the-art semantic understanding, improving top-k chunk selection. Larger chunks captured more comprehensive context, reducing hallucinations and missing info.

2) Helpfulness: Improved noticeably in the second run. Dopeness prompt augmentation probably also improved clarity, tone, or added user-centric phrasing (making output feel more helpful). Larger chunks likely led to better contextual grounding, especially for abstract or longer questions. This mean we're not retriveing correct context, we’re retrieving useful content and wrapping it in user-friendly responses.

3) Dopeness: Jumped from 0-->1 
This was the direct optimization target. We NUsed creative prompt augmentation to inject tone/style. Our Style shift doesn’t hurt correctness because we’ve improved grounding through better retrieval and larger chunks. Our prompts also were more specific for context. 

![label](Homework_Chart.png)